In [ ]:
!pip install Sastrawi
!pip install selenium

In [ ]:
import pandas as pd
import re

from tqdm import tqdm

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
# library untuk proses modeling
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# library untuk evaluasi model
from sklearn.metrics import classification_report, confusion_matrix

# plotting
import matplotlib.pyplot as plt
import seaborn as sns

# save model
import pickle

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def web_driver():
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver

def get_element_text(driver, xpath):
    try:
        return WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, xpath))
        ).text.strip()
    except Exception as e:
        print(f"Error finding element with XPath {xpath}: {e}")
        return ""

def extract_article_content(driver, article_url):
    driver.get(article_url)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//h1'))
    )
    title = get_element_text(driver, './/h1')

    # XPath untuk tanggal
    date = get_element_text(driver, './/p[@class="pt-20 date"]')

    # XPath untuk konten
    content_elements = driver.find_elements(By.XPATH, './/div[@class="news-text"]/p')
    content = " ".join(p.text for p in content_elements)

    # XPath untuk kategori
    kategori = get_element_text(driver, './/div[@class="breadcrumb-content"]/p')

    return {
        "Title": title,
        "Date": date,
        "Content": content,
        "Category": kategori  # Menyimpan kategori dalam hasil
    }

driver = web_driver()

# Meminta input dari pengguna untuk URL artikel
url_input = input("Masukkan URL artikel: ")

# Scraping data dari URL artikel yang diberikan oleh pengguna
article_data = extract_article_content(driver, url_input)

driver.quit()

# Simpan hasil scraping ke dalam DataFrame
df = pd.DataFrame([article_data])  # Mengonversi hasil ke dalam list untuk membuat DataFrame
df.to_csv('hasil_crawling.csv', index=False)

Masukkan URL artikel: https://www.metrotvnews.com/read/NrWC56Mv-tingkatkan-kompetensi-jurnalis-di-indonesia-dewan-pers-apresiasi-bri-fellowship-journalism-2025


In [ ]:
df_acak = df.sample(frac=1).reset_index(drop=True)

In [ ]:
df_acak

,Title,Date,Content,Category
0,"Tingkatkan Kompetensi Jurnalis di Indonesia, D...",Patrick Pinaria • 6 November 2024 20:33,Jakarta: Dewan Pers mengapresiasi program BRI ...,EKONOMI


In [ ]:
def clean_lower(text):
    if isinstance(text, str):
        return text.lower()
    return text

df['lower case'] = df['Content'].apply(clean_lower)

casefolding = pd.DataFrame(df['lower case'])

def clean_punct(text):
    if isinstance(text, str):
        clean_patterns = re.compile(r'[0-9]|[/(){}\[\]\|@,;_]|[^a-z ]')
        text = clean_patterns.sub(' ', text)
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    return text

df['tanda baca'] = df['lower case'].apply(clean_punct)

def _normalize_whitespace(text):
    if isinstance(text, str):
        corrected = re.sub(r'\s+', ' ', text)
        return corrected.strip()
    return text

df['spasi'] = df['tanda baca'].apply(_normalize_whitespace)

def clean_stopwords(text):
    if isinstance(text, str):
        stopword = set(stopwords.words('indonesian'))
        text = ' '.join(word for word in text.split() if word not in stopword)
        return text.strip()
    return text

df['stopwords'] = df['spasi'].apply(clean_stopwords)

def sastrawistemmer(text):
    factory = StemmerFactory()
    st = factory.create_stemmer()
    text = ' '.join(st.stem(word) for word in tqdm(text.split()) if word in text)
    return text

df['stemming'] = df['stopwords'].apply(sastrawistemmer)

100%|██████████| 82/82 [00:03<00:00, 27.11it/s]


In [ ]:
# 2. Load TF-IDF Vectorizer yang sudah disimpan
filename_tfidf = 'tfidf_vectorizer.sav'
tfidf_vectorizer = pickle.load(open(filename_tfidf, 'rb'))

In [ ]:
corpus = df['stemming'].tolist()
x_tfidf = tfidf_vectorizer.transform(corpus)
feature_names = tfidf_vectorizer.get_feature_names_out()
tfidf_df = pd.DataFrame(x_tfidf.toarray(), columns=feature_names)
cat_df = df["Category"]
tfidf_df['Category'] = cat_df.values
tfidf_df = tfidf_df[['Category'] + [col for col in tfidf_df.columns if col != 'Category']]

tfidf_df

,Category,abadi,abbeele,abdullah,abilitysport,abilitysprot,abraham,absen,acara,aceh,...,york,yose,youth,yunani,yusrizal,za,zeki,zian,zona,zuri
0,EKONOMI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# 3. Load Model SVD (LSA) yang sudah dilatih
filename_svd = 'svd_model.sav'
svd = pickle.load(open(filename_svd, 'rb'))

# Terapkan SVD (dimensionality reduction) pada data baru
x_new_lsa = svd.transform(x_tfidf)
train_lsa_df = pd.DataFrame(x_new_lsa, columns=[f'Component_{i+1}' for i in range(x_new_lsa.shape[1])])

train_lsa_df['Category'] = cat_df.values
train_lsa_df = train_lsa_df[['Category'] + [col for col in train_lsa_df.columns if col != 'Category']]
# Tampilkan DataFrame hasil SVD
train_lsa_df

,Category,Component_1,Component_2,Component_3,Component_4,Component_5,Component_6,Component_7,Component_8,Component_9,...,Component_71,Component_72,Component_73,Component_74,Component_75,Component_76,Component_77,Component_78,Component_79,Component_80
0,EKONOMI,0.082452,0.031761,0.037604,0.024344,-0.075668,0.051503,-0.037063,-0.021476,0.014033,...,-0.034009,0.001111,-0.010149,-0.036117,-0.002687,0.011436,-0.022157,-0.016756,0.006636,0.005071


In [ ]:
# menggunakan label_encoder untuk merubah kata menjadi angka
label_encoder = preprocessing.LabelEncoder()
train_lsa_df['Category'] = label_encoder.fit_transform(train_lsa_df['Category'])

train_lsa_df

,Category,Component_1,Component_2,Component_3,Component_4,Component_5,Component_6,Component_7,Component_8,Component_9,...,Component_71,Component_72,Component_73,Component_74,Component_75,Component_76,Component_77,Component_78,Component_79,Component_80
0,0,0.082452,0.031761,0.037604,0.024344,-0.075668,0.051503,-0.037063,-0.021476,0.014033,...,-0.034009,0.001111,-0.010149,-0.036117,-0.002687,0.011436,-0.022157,-0.016756,0.006636,0.005071


In [ ]:
# Load the saved model from file
filename = 'lr_model.sav'
lr_model = pickle.load(open(filename, 'rb'))

In [ ]:
y_test = train_lsa_df['Category']
x_test = train_lsa_df.drop(['Category'], axis=1)
y_pred = lr_model.predict(x_test)

y_pred

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


array([0])

In [ ]:
# melihat nilai actual dan predicted
a = pd.DataFrame({'Actual value': y_test, 'Predicted value':y_pred})
a

,Actual value,Predicted value
0,0,0
